In [1]:
import MeCab
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

'''
# 必要であれば変更してください
import os
os.chdir('/root/userspace/team9_project')
'''
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
tagger = MeCab.Tagger("-Ochasen")
node = tagger.parse("坊主が屏風に上手に坊主の絵を描いた")
print(node)

坊主	ボウズ	坊主	名詞-一般		
が	ガ	が	助詞-格助詞-一般		
屏風	ビョウブ	屏風	名詞-一般		
に	ニ	に	助詞-格助詞-一般		
上手	ジョウズ	上手	名詞-形容動詞語幹		
に	ニ	に	助詞-副詞化		
坊主	ボウズ	坊主	名詞-一般		
の	ノ	の	助詞-連体化		
絵	エ	絵	名詞-一般		
を	ヲ	を	助詞-格助詞-一般		
描い	エガイ	描く	動詞-自立	五段・カ行イ音便	連用タ接続
た	タ	た	助動詞	特殊・タ	基本形
EOS



In [3]:
def tokenize(sentence):
    """日本語の文を形態素の列に分割する関数

    :param sentence: str, 日本語の文
    :return tokenized_sentence: list of str, 形態素のリスト
    """
    node = tagger.parse(sentence)
    node = node.split("\n")
    tokenized_sentence = []
    for i in range(len(node)):
        feature = node[i].split("\t")
        if feature[0] == "EOS":
            # 文が終わったら終了
            break
        # 分割された形態素を追加
        tokenized_sentence.append(feature[0])
    return tokenized_sentence

In [4]:
import pandas as pd

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [5]:
def load_data(path):
    """『こころ』を読み込むための関数

    :param path: str, 『こころ』のパス
    :return text: list of list of str, 各文がトークナイズされた『こころ』
    """
    text = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            line = tokenize(line)
            text.append(line)
    return text

In [6]:
def data_tokenize(df):
    text=[]
    for recipe in df:
        word = tokenize(recipe)
        text.append(word)
    return text

In [7]:
text = load_data("dataset/Material/recipe1M/recipe1M_0.txt")
#text = load_data("dataset/Material/recipes2.txt")
#text = load_data("dataset/kokoro.txt")

AttributeError: 'NoneType' object has no attribute 'split'

In [195]:
text[100]

['ボウル',
 'に',
 '粉',
 'と',
 'ベーキングパウダー',
 'を',
 '入れ',
 'て',
 '混ぜ合わ',
 'せる',
 '。',
 '1',
 'に',
 'オリーブ',
 'オイル',
 'を',
 '加え',
 'て',
 '手',
 'で',
 '混ぜる',
 '。',
 '水',
 'を',
 'まず',
 '半量',
 '加え',
 'て',
 'よく',
 '混ぜ',
 '、',
 '残り',
 'の',
 '半量',
 'を',
 '加え',
 'たら',
 '手',
 'に',
 '付か',
 'なく',
 'なる',
 'まで',
 'よく',
 '混ぜる',
 '。',
 '全体',
 'を',
 'まとめ',
 'て',
 '天',
 '板',
 'の',
 '大き',
 'さ',
 'に',
 '綿棒',
 'で',
 'のばす',
 '。',
 'クッキング',
 'シート',
 'の',
 '上',
 'に',
 '生地',
 'を',
 'のせ',
 'ピザ',
 'ソース',
 'の',
 '材料',
 'を',
 '混ぜ合わ',
 'せ',
 '伸ばし',
 'た',
 '生地',
 'に',
 '塗る',
 '。',
 '好み',
 'の',
 '具',
 '材',
 'を',
 'のせる',
 '。',
 'オリーブ',
 'オイル',
 '(',
 '分量',
 '外',
 ')',
 'を',
 'かけ',
 '、',
 '余熱',
 'あり',
 '250',
 '℃',
 '以上',
 '(',
 '我が家',
 'は',
 '300',
 '℃)',
 'で',
 '10',
 '〜',
 '15',
 '分',
 '焼い',
 'て',
 '出来上がり',
 '♪',
 '生地',
 'に',
 'いり',
 'ゴマ',
 'を',
 '混ぜ',
 '、',
 'オリーブ',
 'オイル',
 'を',
 'ゴマ油',
 'に',
 '変更',
 '♪',
 'ドライ',
 'カレー',
 'を',
 'のっけ',
 'て',
 'ウマウマ',
 '(^¬^)♪']

In [149]:
# 特殊なトークンとそのIDは事前に定義しておきます。
PAD_TOKEN = '<PAD>' # あとで説明するpaddingに使います
UNK_TOKEN = '<UNK>' # 辞書にない単語は全てこのUNKトークンに置き換えます。(UNKの由来はunkownです)
PAD = 0 # <PAD>のID
UNK = 1 # <UNK>のID

In [150]:
# 辞書の初期化
word2id = {
    PAD_TOKEN: PAD,
    UNK_TOKEN: UNK,
}

# 辞書に含める単語の最低出現回数
# 今回はコーパスのサイズが小さいので、全ての単語を辞書に含めることにします
MIN_COUNT = 1

In [151]:
class Vocab(object):
    """語彙を管理するためのクラス"""
    def __init__(self, word2id={}):
        """
        :param word2id: 単語(str)をインデックス(int)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    

    def build_vocab(self, sentences, min_count=1):
        """コーパスから語彙の辞書を構築するメソッド

        :param sentences: list of list of str, コーパス
        :param min_count: int, 辞書に含める単語の最小出現回数
        """
        # 各単語の出現回数をカウントする辞書を作成します
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加えます
        # 出現回数の高い単語から順にword2idに追加していきます
        # 出現回数に-1をかけた値でsortすることで出現回数の降順になるようにしています
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word

        # 語彙に含まれる単語の出現回数を保持します（あとで使います）
        self.raw_vocab = {w: word_counter[w] for w in self.word2id.keys() if w in word_counter}

In [152]:
vocab = Vocab(word2id=word2id)
vocab.build_vocab(text, min_count=MIN_COUNT)
print("語彙数:", len(vocab.word2id))

語彙数: 7923


In [153]:
def sentence_to_ids(vocab, sen):
    """
    単語のリストをIDのリストに変換する関数

    :param vocab: class `Vocab` object
    :param sen: list of str, 文を分かち書きして得られた単語のリスト
    :return out: list of int, 単語IDのリスト
    """
    out = [vocab.word2id.get(word, UNK) for word in sen] # 辞書にない単語にはUNKのIDを割り振ります
    return out

In [154]:
# 日本語のテキストを単語IDに変換します。
id_text = [sentence_to_ids(vocab, sen) for sen in text]

In [155]:
print(text[0])
print(id_text[0])

['こころ']
[4150]


In [156]:
def pad_seq(seq, max_length):
    """Paddingを行う関数

    :param seq: list of int, 単語のインデックスのリスト
    :param max_length: int, バッチ内の系列の最大長
    :return seq: list of int, 単語のインデックスのリスト
    """
    seq += [PAD for i in range(max_length - len(seq))]
    return seq

## CBOW

In [164]:
# Hyper Parameters
batch_size = 64 # ミニバッチのサイズ
n_batches = 900 # 今回学習するミニバッチの数
vocab_size = len(vocab.word2id) # 語彙の総数
embedding_size = 600 # 各単語に割り当てるベクトルの次元数

In [165]:
class TestIter(object):
    def __init__(self):
        self.iter = 0
        self.max_iter = 5
    
    def __iter__(self): # 必須
        print("iter関数が呼び出されました")
        return self
    
    def __next__(self):
        self.iter += 1
        print("next関数が呼び出されました({}回目)".format(self.iter))
        if self.iter < self.max_iter:
            return None
        else:
            print("max_iterに達したので終了します")
            raise StopIteration

In [166]:
class DataLoaderCBOW(object):
    """CBOWのデータローダー"""
    def __init__(self, text, batch_size, window=3):
        """
        :param text: list of list of int, 単語をIDに変換したデータセット
        :param batch_size: int, ミニバッチのサイズ
        :param window: int, 周辺単語とターゲットの単語の最大距離
        """
        self.text = text
        self.batch_size = batch_size
        self.window = window
        self.s_pointer = 0 # データセット上を走査する文単位のポインタ
        self.w_pointer = 0 # データセット上を走査する単語単位のポインタ
        self.max_s_pointer = len(text) # データセットに含まれる文の総数

    def __iter__(self):
        return self

    def __next__(self):
        batch_X = []
        batch_Y = []
        while len(batch_X) < self.batch_size:
            # 走査する対象の文
            sen = self.text[self.s_pointer]
            
            # 予測すべき単語
            word_Y = sen[self.w_pointer]
            
            # 入力となる単語群を取得
            start = max(0, self.w_pointer - self.window)
            word_X = sen[start:self.w_pointer] + \
                sen[self.w_pointer + 1:self.w_pointer + self.window + 1]
            word_X = pad_seq(word_X, self.window * 2)
            
            batch_X.append(word_X)
            batch_Y.append(word_Y)
            self.w_pointer += 1
            
            if self.w_pointer >= len(sen):
                # 文を走査し終わったら次の文の先頭にポインタを移行する
                self.w_pointer = 0
                self.s_pointer += 1
                if self.s_pointer >= self.max_s_pointer:
                    # 全ての文を走査し終わったら終了する
                    self.s_pointer = 0
                    raise StopIteration

        # データはtorch.Tensorにする必要があります。dtype, deviceも指定します。
        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device)
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device)

        return batch_X, batch_Y

In [167]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(CBOW, self).__init__()
        """
        :param vocab_size: int, 語彙の総数
        :param embedding_size: int, 単語埋め込みベクトルの次元
        """
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        # 埋め込み層
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        # 全結合層(バイアスなし)
        self.linear = nn.Linear(self.embedding_size, self.vocab_size, bias=False)
        
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, batch_X, batch_Y):
        """
        :param batch_X: torch.Tensor(dtype=torch.long), (batch_size, window*2)
        :param batch_Y: torch.Tensor(dtype=torch.long), (batch_size,)
        :return loss: torch.Tensor(dtype=torch.float), CBOWのloss
        """
        emb_X = self.embedding(batch_X) # (batch_size, window*2, embedding_size)
        # paddingした部分を無視するためにマスクをかけます
        emb_X = emb_X * (batch_X != PAD).float().unsqueeze(-1) # (batch_size, window*2, embedding_size)
        sum_X = torch.sum(emb_X, dim=1) # (batch_size, embedding_size)
        lin_X = self.linear(sum_X) # (batch_size, vocab_size)
        log_prob_X = F.log_softmax(lin_X, dim=-1) # (batch_size, vocab_size)
        loss = F.nll_loss(log_prob_X, batch_Y)
        return loss

In [168]:
# モデル
cbow = CBOW(vocab_size, embedding_size).to(device) # iLectで実行する場合warning (GPU is too old) が出ますが, 実行に問題はないので気にせず進めてください.
# optimizer
optimizer_cbow = optim.Adam(cbow.parameters())
# データローダー
dataloader_cbow = DataLoaderCBOW(id_text, batch_size)

In [169]:
def compute_loss(model, inputs, optimizer=None, is_train=True):
    """lossを計算するための関数
    
    is_train=Trueならモデルをtrainモードに、
    is_train=Falseならモデルをevaluationモードに設定します
    
    :param model: 学習させるモデル
    :param inputs: モデルへの入力
    :param optimizer: optimizer
    :param is_train: bool, モデルtrainさせるか否か
    """
    model.train(is_train)

    # lossを計算します。
    loss = model(*inputs)

    if is_train:
        # .backward()を実行する前にmodelのparameterのgradientを全て0にセットします
        optimizer.zero_grad()
        # parameterのgradientを計算します。
        loss.backward()
        # parameterのgradientを用いてparameterを更新します。
        optimizer.step()

    return loss.item()

In [170]:
start_at = time.time()

for batch_id, (batch_X, batch_Y) in enumerate(dataloader_cbow):
    loss = compute_loss(cbow, (batch_X, batch_Y), optimizer=optimizer_cbow, is_train=True)
    if batch_id % 100 == 0:
        print("batch:{}, loss:{:.4f}".format(batch_id, loss))
    if batch_id >= n_batches:
        break

end_at = time.time()

print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

batch:0, loss:8.9773
batch:100, loss:5.8359
batch:200, loss:5.2109
batch:300, loss:6.9613
batch:400, loss:5.2537
batch:500, loss:5.4618
batch:600, loss:6.5011
batch:700, loss:5.0042
batch:800, loss:5.5086
batch:900, loss:5.8533
Elapsed time: 7.53 [sec]


In [171]:
# 埋め込み層のパラメータのみを保存する
torch.save(cbow.embedding.weight.data.cpu().numpy(),  "kokoro_cbow_embedding_{}b_min_{}_dim_{}.pth".format(n_batches,MIN_COUNT,embedding_size))
#torch.save(cbow.embedding.weight.data.cpu().numpy(),  "recipe_cbow_embedding_{}b_min_{}_dim_{}.pth".format(n_batches,MIN_COUNT,embedding_size))

## Skipgram

In [172]:
class DataLoaderSG(object):
    """Skipgramのためのデータローダー"""
    def __init__(self, text, batch_size, window=3):
        """
        :param text: list of list of int, 単語をIDに変換したデータセット
        :param batch_size: int, ミニバッチのサイズ
        :param window: int, 周辺単語と入力単語の最大距離
        """
        self.text = text
        self.batch_size = batch_size
        self.window = window
        self.s_pointer = 0 # データセット上を走査する文単位のポインタ
        self.w_pointer = 0 # データセット上を走査する単語単位のポインタ
        self.max_s_pointer = len(text) # データセットに含まれる文の総数

    def __iter__(self):
        return self

    def __next__(self):
        batch_X = []
        batch_Y = []

        while len(batch_X) < self.batch_size:
            sen = self.text[self.s_pointer]
            
            # Skipgramでは入力が1単語
            word_X = sen[self.w_pointer]

            # 出力は周辺単語
            start = max(0,self.w_pointer-self.window)
            word_Y = sen[start:self.w_pointer]+sen[self.w_pointer+1:self.w_pointer+self.window+1]
            word_Y = pad_seq(word_Y,self.window*2)

            batch_X.append(word_X)
            batch_Y.append(word_Y)
            self.w_pointer += 1

            if self.w_pointer >= len(sen):
                self.w_pointer = 0
                self.s_pointer += 1
                if self.s_pointer >= self.max_s_pointer:
                    self.s_pointer = 0
                    raise StopIteration

        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device)
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device)

        return batch_X, batch_Y

In [173]:
class Skipgram(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        """
        :param vocab_size: int, 語彙の総数
        :param embedding_size: int, 単語埋め込みベクトルの次元
        """
        super(Skipgram, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        self.linear = nn.Linear(self.embedding_size, self.vocab_size, bias=False)

    def forward(self, batch_X, batch_Y):
        """
        :param batch_X: torch.Tensor(dtype=torch.long), (batch_size,)
        :param batch_Y: torch.Tensor(dtype=torch.long), (batch_size, window*2)
        :return loss: torch.Tensor(dtype=torch.float), Skipgramのloss
        """
        emb_X = self.embedding(batch_X)
        lin_X = self.linear(emb_X)
        log_prob_X = F.log_softmax(lin_X,dim=-1)
        log_prob_X = torch.gather(log_prob_X, 1, batch_Y) # (batch_size, window*2)
        # paddingした単語のlossは計算しないようにマスクをかけます(=lossの該当部分を0にします)
        log_prob_X = log_prob_X * (batch_Y != PAD).float() # (batch_size, window*2)
        loss = log_prob_X.sum(1).mean().neg()
        return loss

In [174]:
sg = Skipgram(vocab_size, embedding_size).to(device)
optimizer_sg = optim.Adam(sg.parameters())
dataloader_sg = DataLoaderSG(id_text, batch_size)

In [175]:
start_at = time.time()
for batch_id, (batch_X, batch_Y) in enumerate(dataloader_sg):
    loss = compute_loss(sg, (batch_X, batch_Y), optimizer=optimizer_sg, is_train=True)
    if batch_id % 100 == 0:
        print("batch:{}, loss:{:.4f}".format(batch_id, loss))
    if batch_id >= n_batches:
        break
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

batch:0, loss:48.2575
batch:100, loss:38.6715
batch:200, loss:34.5417
batch:300, loss:45.8497
batch:400, loss:36.2331
batch:500, loss:32.9121
batch:600, loss:45.5428
batch:700, loss:33.5448
batch:800, loss:37.3178
batch:900, loss:42.0639
Elapsed time: 7.37 [sec]


In [176]:
# 埋め込み層のパラメータのみを保存する
torch.save(sg.embedding.weight.data.cpu().numpy(),  "kokoro_sg_embedding_{}b_min_{}_dim_{}.pth".format(n_batches,MIN_COUNT,embedding_size))
#torch.save(sg.embedding.weight.data.cpu().numpy(),  "recipe_sg_embedding_{}b_min_{}_dim_{}.pth".format(n_batches,MIN_COUNT,embedding_size))

## 4. Skipgram with Negative Sampling

In [177]:
# negative samplingに使う確率分布
weights = np.power([0, 0] + list(vocab.raw_vocab.values()), 0.75)
weights = weights / weights.sum()

In [178]:
class DataLoaderSGNS(object):
    def __init__(self, text, batch_size, window=3, n_negative=5, weights=None):
        """
        :param text: list of list of int, 単語をIDに変換したデータセット
        :param batch_size: int, ミニバッチのサイズ
        :param window: int, 周辺単語と入力単語の最大距離
        :param n_negative: int, 負例の数
        :param weights: numpy.ndarray, Negative Samplingで使う確率分布
        """
        self.text = text
        self.batch_size = batch_size
        self.window = window
        self.n_negative = n_negative
        self.weights = None
        if weights is not None:
            self.weights = torch.FloatTensor(weights) # negative samplingに使う確率分布
        self.s_pointer = 0 # 文のポインタ
        self.w_pointer = 0 # 単語のポインタ
        self.max_s_pointer = len(text)

    def __iter__(self):
        return self
    
    def __next__(self):
        batch_X = []
        batch_Y = []
        batch_N = [] # 負例
        while len(batch_X) < self.batch_size:
            sen = self.text[self.s_pointer]
            start = max(0, self.w_pointer - self.window)
            word_X = sen[self.w_pointer]
            word_Y = sen[start:self.w_pointer] + \
                sen[self.w_pointer + 1:self.w_pointer + self.window + 1]
            word_Y = pad_seq(word_Y, self.window * 2)
            batch_X.append(word_X)
            batch_Y.append(word_Y)

            # 多項分布で負例をサンプリング
            # 実装を簡略化するために、正例の除去は行っていません
            negative_samples = torch.multinomial(self.weights, self.n_negative) # (n_negative,)
            batch_N.append(negative_samples.unsqueeze(0)) # (1, n_negative)

            self.w_pointer += 1
            if self.w_pointer >= len(sen):
                self.w_pointer = 0
                self.s_pointer += 1
                if self.s_pointer >= self.max_s_pointer:
                    self.s_pointer = 0
                    raise StopIteration

        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device)
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device)
        batch_N = torch.cat(batch_N, dim=0).to(device) # (batch_size, n_negative)

        return batch_X, batch_Y, batch_N

In [179]:
class SGNS(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        """
        :param vocab_size: int, 語彙の総数
        :param embedding_size: int, 単語埋め込みベクトルの次元
        """
        super(SGNS, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        # 入力単語の埋め込み層
        self.i_embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        # 出力単語の埋め込み層
        self.o_embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        
        nn.init.xavier_uniform_(self.i_embedding.weight)
        nn.init.xavier_uniform_(self.o_embedding.weight)

    def forward(self, batch_X, batch_Y, batch_N):
        """
        :param batch_x: torch.Tensor(dtype=torch.long), (batch_size,)
        :param batch_y: torch.Tensor(dtype=torch.long), (batch_size, window*2)
        :param batch_n: torch.Tensor(dtype=torch.long), (batch_size, n_negative)
        """
        embed_X = self.i_embedding(batch_X).unsqueeze(2) # (batch_size, embedding_size, 1)
        embed_Y = self.o_embedding(batch_Y) # (batch_size, window*2, embedding_size)
        embed_N = self.o_embedding(batch_N).neg() # (batch_size, n_negative, embedding_size)
        loss_Y = torch.bmm(embed_Y, embed_X).squeeze().sigmoid().log() # (batch_size, window*2)
        loss_Y = loss_Y * (batch_Y != PAD).float() # (batch_size, window*2)
        loss_Y = loss_Y.sum(1) # (batch_size,)
        loss_N = torch.bmm(embed_N, embed_X).squeeze().sigmoid().log().sum(1) # (batch_size,)
        return -(loss_Y + loss_N).mean()

In [180]:
sgns = SGNS(vocab_size, embedding_size).to(device)
optimizer_sgns = optim.Adam(sgns.parameters())
dataloader_sgns = DataLoaderSGNS(id_text, batch_size, n_negative=5, weights=weights)
start_at = time.time()
for batch_id, (batch_X, batch_Y, batch_N) in enumerate(dataloader_sgns):
    loss = compute_loss(sgns, (batch_X, batch_Y, batch_N), optimizer=optimizer_sgns, is_train=True)
    if batch_id % 100 == 0:
        print("batch:{}, loss:{:.4f}".format(batch_id, loss))
    if batch_id >= n_batches:
        break
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

batch:0, loss:7.0823
batch:100, loss:7.0294
batch:200, loss:6.5883
batch:300, loss:7.3731
batch:400, loss:6.7110
batch:500, loss:6.6285
batch:600, loss:7.2633
batch:700, loss:6.3686
batch:800, loss:6.7104
batch:900, loss:7.1610
Elapsed time: 40.13 [sec]


In [181]:
# Embeddingのパラメータのみを保存する
torch.save(sgns.i_embedding.weight.data.cpu().numpy(),  "kokoro_sgns_embedding_{}b_min_{}_dim_{}.pth".format(n_batches,MIN_COUNT,embedding_size))
#torch.save(sgns.i_embedding.weight.data.cpu().numpy(),  "recipe_sgns_embedding_{}b_min_{}_dim_{}.pth".format(n_batches,MIN_COUNT,embedding_size))

In [185]:
def compute_word_similarity(embedding_path, word, n):
    """
    与えられた単語に最も似ている単語とcos類似度を返す関数

    :param embedding_path: str, 保存した埋め込み層のパラメータのパス
    :param word: str, 単語
    :param n: int
    :return out: str, 上位n個の類似単語とそのcos類似度
    """
    embedding = torch.load(embedding_path)

    # 単語ベクトルを全て単位ベクトルにする
    norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
    norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
    embedding /= norm
    e = embedding[vocab.word2id[word]]

    # 単語ベクトル同士のcos類似度を計算する
    cos_sim = np.dot(embedding, e.reshape(-1, 1)).reshape(-1,)
    most_sim = np.argsort(cos_sim)[::-1][1:n+1] # 自分は除く
    most_sim_words = [vocab.id2word[_id] for _id in most_sim]
    top_cos_sim = cos_sim[most_sim]
    out = ", ".join([w+"({:.4f})".format(v) for w, v in zip(most_sim_words, top_cos_sim)])
    return out

In [187]:
# kokoro
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_similarity(
        "kokoro_"+model + "_embedding_900b_min_1_dim_600.pth", "私", 5))

cbow	: 父(0.6770), 奥さん(0.6755), 彼ら(0.6735), 医者(0.6470), あなた(0.6346)
sg	: 形(0.1762), ずらし(0.1708), 少ない(0.1474), 弱く(0.1407), 動き出し(0.1378)
sgns	: あなた(0.6294), はたして(0.5896), 先生(0.5766), しかし(0.5613), 奥さん(0.5569)


In [184]:
# 300バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_similarity(
        "recipe_"+model + "_embedding_300b_min_3.pth", "牛肉", 5))

FileNotFoundError: [Errno 2] No such file or directory: 'kokoro_cbow_embedding_300b_min_3.pth'

In [95]:
# 900バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_similarity(
        "recipe_"+ model + "_embedding_900b_min_2_dim_600.pth", "牛肉", 5))

cbow	: ひとつまみ(0.8166), 青(0.8139), さめ(0.8124), 残し(0.8072), 流し込む(0.8056)
sg	: 煮え(0.1487), やつ(0.1441), 振り掛ける(0.1436), マグネシウム(0.1391), 鳴っ(0.1371)
sgns	: おい(0.9195), みじん切り(0.9177), 目(0.9169), 袋(0.9164), 酢(0.9164)
